## Hog Face

The first step of recognition find all faces in the picture by face detection. Such as iPhones camera application. In this project, we use Histogram of Oriented Gradients (HOG) method. The principle behind the Hog: 
     1.we need to convert rgb picture to black and white. 2.Then we'll look at each pixel in the image. For a single pixel, we also look at the other pixels around it. 3.Our goal is to find the pixel directly. And then we're going to draw an arrow that's going to indicate the direction that the image is going to go dark. 



In [1]:
import sys
import dlib
from skimage import io

We could use the dlib's face_detector which has the fuction of Hog face.

In [2]:

file_name = '/Users/mac/7390/Final/faces/Emma.jpg'
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
win = dlib.image_window()
# Load the image into an array
image = io.imread(file_name )
# Run the HOG face detector on the image data.
# The result will be the bounding boxes of the faces in our image.
detected_faces = face_detector(image, 1)

print("I found {} faces in the file {}".format(len(detected_faces), file_name))


I found 1 faces in the file /Users/mac/7390/Final/faces/Emma.jpg


If you want to create image_window, you must make sure there you have set up X11 in your laptop to show the image imread.

In [3]:
# Open a window on the desktop showing the image
win.set_image(image)


In [4]:
	# Detected faces are returned as an object with the coordinates 
	# of the top, left, right and bottom edges
    
    # Loop through each face we found in the image
for i, face_rect in enumerate(detected_faces):
    print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))


- Face #0 found at Left: 91 Top: 246 Right: 553 Bottom: 708


The detect result is shown above.

In [5]:
	# Draw a box around each face we found
	win.add_overlay(face_rect)
	        

In [6]:
# Wait until the user hits <enter> to close the window	        
dlib.hit_enter_to_continue()

## 68 landmark


   Humans can easily recognize that both images are the same people, but the computer thinks these two pictures are two completely different people. To solve this, we will try to distort each image so that the eyes and lips are always in the sample place in the image. This will make it easier to compare the differences between faces in the next steps. To do this, we'll use an algorithm called face landmark estimation.

The basic idea of this algorithm is to find the specific points. Usually, we find 68 common points(known as landmarks)


In [11]:
import sys
import dlib
import os
from skimage import io

There is a muture 68_face_landmarks model we could use which uses the principle of ERT（ensemble of regression trees.That is, a regression tree method based on gradients to improve learning. The algorithm uses cascading regression factors. First, a series of calibrated face images are used as the training set. Then a model is generated.

When an image is obtained, the algorithm generates an initial shape by first estimating a rough feature point location, and then using the gradient boosting algorithm to reduce the sum of the squared errors of the initial shape and ground truth. The least squares method was used to minimize the error and a cascade regression factor for each stage was obtained.

In [15]:
current_path = os.getcwd()
predictor_path = current_path + "/model/shape_predictor_68_face_landmarks (1).dat"
#predictor_model = "shape_predictor_68_face_landmarks (1).dat"
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
face_pose_predictor = dlib.shape_predictor(predictor_path)
win = dlib.image_window()
# Take the image file name from the command line
file_name = '/Users/mac/7390/Final/faces/Emma.jpg'
# Load the image
image = io.imread(file_name)

In [16]:
# Run the HOG face detector on the image data
detected_faces = face_detector(image, 1)
print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

Found 1 faces in the image file /Users/mac/7390/Final/faces/Emma.jpg


In [17]:
# Show the desktop window with the image
win.set_image(image)


The tasks above are similar to those in step1.

In [18]:
# Loop through each face we found in the image
for i, face_rect in enumerate(detected_faces):

	# Detected faces are returned as an object with the coordinates 
	# of the top, left, right and bottom edges
    print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))
pose_landmarks = face_pose_predictor(image, face_rect)
print("Part 0: {}, Part 1: {} ...".format(pose_landmarks.part(0),
                                                  pose_landmarks.part(1)))

- Face #0 found at Left: 91 Top: 246 Right: 553 Bottom: 708
Part 0: (190, 460), Part 1: (187, 500) ...


In [19]:
# Draw a box around each face we found
win.add_overlay(face_rect)

Get the landmarks.

In [9]:

pose_landmarks = face_pose_predictor(image, face_rect)


Every landmarks within range 68 has their specific meaning, like point 30 represents the tip of nose, and similarly, the landmark 27 represents the root of the nose.

In [10]:
# tip of the nose  30
# root of the nose 27  
# chin  8  
# left eye outer corner 36  
# left eye inner corner 39  
# right eye outer corner 45  
# right eye inner corner 42  
# middle of mouth   66  
# mouth left corner   48  
# mouth right corner  54  
# left face outer 0  
# right face outer 16  
#pose_landmarks.part(8),pose_landmarks.part(36),pose_landmarks.part(39),pose_landmarks.part(45),pose_landmarks.part(42),pose_landmarks.part(66),pose_landmarks.part(48),pose_landmarks.part(54),pose_landmarks.part(0),pose_landmarks.part(16)
print(pose_landmarks.part(30))
print(pose_landmarks.part(27))
print(pose_landmarks.part(8))
print(pose_landmarks.part(36))
print(pose_landmarks.part(39))
print(pose_landmarks.part(45))
print(pose_landmarks.part(42))
print(pose_landmarks.part(66))
print(pose_landmarks.part(48))
print(pose_landmarks.part(54))
print(pose_landmarks.part(0))
print(pose_landmarks.part(16))

(65, 223)
(82, 173)
(81, 322)
(47, 165)
(74, 173)
(155, 181)
(117, 180)
(76, 265)
(58, 259)
(126, 266)
(38, 179)
(236, 183)


In [11]:
# Draw the face landmarks on the screen.
win.add_overlay(pose_landmarks)

In [20]:
dlib.hit_enter_to_continue()


## Picture Transformation

When we get a face, it could be front face but it could also be a side face. So if we use the landmarks directly gotten from 68 landmarks model, it will affect the training in following step. Because of this, we should try to twist the landmarks in order to revise some image in the circumatance like side face or any other.

In [24]:
import sys
import dlib
import cv2
import os
import openface

In [25]:
current_path = os.getcwd()
predictor_path = current_path + "/model/shape_predictor_68_face_landmarks (1).dat"
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
face_pose_predictor = dlib.shape_predictor(predictor_path)
face_aligner = openface.AlignDlib(predictor_path)
# Take the image file name from the command line
file_name = '/Users/mac/7390/Final/faces/Emma.jpg'
# Load the image
image = cv2.imread(file_name)

In [26]:
# Run the HOG face detector on the image data
detected_faces = face_detector(image, 1)
print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

Found 1 faces in the image file /Users/mac/7390/Final/faces/Emma.jpg


The face_aligner.align is the tool to revise the landmarks got from 68_landmarks_model.

In [27]:
# Loop through each face we found in the image
for i, face_rect in enumerate(detected_faces):

	# Detected faces are returned as an object with the coordinates 
	# of the top, left, right and bottom edges
	print("- Face #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))

	# Get the the face's pose
	pose_landmarks = face_pose_predictor(image, face_rect)

	# Use openface to calculate and perform the face alignment
	alignedFace = face_aligner.align(534, image, face_rect, landmarkIndices=openface.AlignDlib.OUTER_EYES_AND_NOSE)

	# Save the aligned image to a file
	cv2.imwrite("aligned_face_{}.jpg".format(i), alignedFace)

- Face #0 found at Left: 91 Top: 246 Right: 553 Bottom: 708


## License

Copyright <2018> <Haoran Han & Lintong Yang>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.